# Projeto 2-1: Estabilização de imagens

### Guilherme Zaborowsky Graicer

Motivação: https://www.youtube.com/watch?v=4vt7bGEen2s

##### Agora que você consegue capturar o fluxo ótico entre duas imagens consecultivas, vamos utilizá-lo de modo inverso: como seria uma forma de compensar eventuais oscilações na câmera?


Para compensar as eventuais oscilações na câmera, pode-se calcular o fluxo ótico entre o frame atual e o primeiro frame do vídeo e transladar o vídeo. A quantidade dessa translação é igual a da média do fluxo ótico dos pontos onde foi calculado, e o sentido é oposto.

##### Projete um programa que captura as imagens da webcam e realiza a estabilização da imagem. Você notará que se utilizar o programa acima, a estabilização será parcial e falha. Por que?

Isso foi feito, mas não está mais aqui pois foi substituido pelo algoritmo com o Dense Optical Flow. Sim, a estabilização estava parcial e falha. Isso acontece pois ela calcula o fluxo ótico de pontos escolhidos como notáveis e esses pontos podem estar tanto no centro do video quanto nas bordas.

###### Você deve construir um Jupyter Notebook que utiliza o Dense Optical Flow para corrigir o problema acima. O notebook deve conter comentários acerca da solução usada.

A ideia é usar o Dense Optical Flow, que calcula o fluxo de todos os pixels da imagem dada, porém usando como parâmetro para fazer esse cálculo um corte do centro do frame.

In [2]:
%reset -f
%matplotlib inline
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import time

In [26]:
captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)


avg_x = 0
avg_y = 0

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

try:
    while(1):

        ret, previous = captura.read()
        previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)
        
        ret, actual = captura.read()

        actual_gray = cv.cvtColor(actual, cv.COLOR_BGR2GRAY)

        # Calcula o Fluxo Otico do frame atual com o frame anterior
        
        flow = cv.calcOpticalFlowFarneback(previous_gray[200:280, 280:360], actual_gray[200:280, 280:360], None, 0.5, 3, 15, 3, 5, 1.2, 0)
 
        
        # Calcula a media do fluxo optico em x e y (e soma ela ao fluxo total) 
            
        avg_x += np.average(flow[:,:,0])
        avg_y += np.average(flow[:,:,1])
        
        
        
        # Translada a imagem a mesma quantidade da media do fluxo optico e sentido contrário (em ambas direções)
        
        rows, cols, cor = actual.shape

        M = np.array([[1, 0, -avg_x], [0, 1, -avg_y]], dtype=np.float32)
        actual = cv.warpAffine(actual, M, (cols,rows))
        
        # Mostra a imagem transladada
        cv.imshow("Video", actual)
    
        # Atualiza a imagem anterior com a imagem atual e copia os pontos.
        previous_gray = actual_gray.copy()

        # Pressione ESC para sair do loop
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    
except cv.error as e:
   pass

captura.release()
cv.destroyAllWindows()
